In [1]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import warnings
import xarray

import sklearn.metrics
import sklearn.model_selection
import scipy.stats

In [3]:
FOLDER_DATA = pathlib.Path("../Data/Task1/Original/")
FILE_TWEETS = "/home/kristian/Projects/a2/data/tweets/2020_tweets_rain_sun_vocab_emojis_locations_bba_Tp_era5_no_bots_normalized_filtered.nc"
FILE_TWEETS = "/p/scratch/deepacf/maelstrom/maelstrom_data/ap2/data/tweets/2017_2020_tweets_rain_sun_vocab_emojis_locations_bba_Tp_era5_no_bots_normalized_filtered_weather_stations_fix_predicted_simpledeberta_radar.nc"
FOLDER_TWEETS_MODEL_DATA = pathlib.Path("/p/scratch/deepacf/maelstrom/maelstrom_data/ap2/data/UQ-PLM//Task5/Original/")

In [4]:
!mkdir -p $FOLDER_TWEETS_MODEL_DATA

In [ ]:
!cp /p/scratch/deepacf/maelstrom/maelstrom_data/ap2/data/UQ-PLM//Task1/Original/test_out.pkl $FOLDER_TWEETS_MODEL_DATA

In [15]:
def load_pickle(filename):
    with open(filename, "rb") as input_file:
        return pickle.load(input_file)  

In [ ]:
for filename in os.listdir(FOLDER_DATA):
    data = load_pickle(filename=FOLDER_DATA / filename)
    print(f"{filename=}\n{data[0]}\n{data[-1]}")
type(data)

In [16]:
tweets = xarray.load_dataset(FILE_TWEETS)

In [21]:
tweets.text_normalized.values, tweets.text_normalized.values.shape

(array(['still got my lightning ar that is it tho',
        'Another lap around the sun. Good work people.',
        'Happy sun with face sun with face sun with face', ...,
        'It is the reverse of being draped over a sun lounger',
        'A very picturesque light dusting of snow plus is creating a thrilling atmosphere. I am in bed, cozy under my duvet with dogs on top of the duvet.',
        'This is the most surreal NYE that I have ever known. Not a car track in the snow at nearly'],
       dtype=object),
 (1218609,))

In [18]:
np.all([x==y for x, y in zip(tweets.tp_h.values>1e-6, tweets.raining.values)])

True

In [19]:
def save_pickle(filename, to_save):
    os.makedirs(filename.parent, exist_ok=True)
    with open(filename, 'wb') as handle:
        pickle.dump(to_save, handle)


In [20]:
X = tweets.text_normalized.values
y = tweets.raining.values

In [22]:
indices_train, indices_test_dev = sklearn.model_selection.train_test_split(
    np.arange(tweets["index"].shape[0]),
    test_size=0.4,
    random_state=42,
    shuffle=True,
    stratify=tweets.raining.values,
)

indices_dev, indices_test = sklearn.model_selection.train_test_split(
    indices_test_dev,
    test_size=0.5,
    random_state=42,
    shuffle=True,
    stratify=tweets.raining.values[indices_test_dev],
)


In [25]:
def convert_features_labels_to_list(features, labels):
    return [[f, l] for f, l in zip(features, labels)]
    
    
for indices, filename in zip([indices_train, indices_dev, indices_test], ['train.pkl', 'dev.pkl', 'test_in.pkl']):
    save_pickle(FOLDER_TWEETS_MODEL_DATA / filename, convert_features_labels_to_list(X[indices], y[indices]))

In [27]:
X.shape

(1218609,)